In [7]:
# coding: utf-8

# In[ ]:


import requests             
from bs4 import BeautifulSoup 
import csv                  
import webbrowser
import io

def display(content, filename='output.html'):
    with open(filename, 'wb') as f:
        f.write(content)
        webbrowser.open(filename)

def get_soup(session, url, show=False):
    r = session.get(url)
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[get_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def post_soup(session, url, params, show=False):
    '''Read HTML from server and convert to Soup'''

    r = session.post(url, data=params)
    
    if show:
        display(r.content, 'temp.html')

    if r.status_code != 200: # not OK
        print('[post_soup] status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')
    
def scrape(url, lang='ALL'):

    # create session to keep all cookies (etc.) between requests
    session = requests.Session()

    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0',
    })


    items = parse(session, url + '?filterLang=' + lang)

    return items

def parse(session, url):
    '''Get number of reviews and start getting subpages with reviews'''

    print('[parse] url:', url)

    soup = get_soup(session, url)

    if not soup:
        print('[parse] no soup:', url)
        return

    #num_reviews = soup.find('span', class_='reviews_header_count').text # get text
    num_reviews = soup.find('span', class_='reviews_header_count')
    num_reviews = num_reviews.text
    num_reviews = num_reviews[1:-1] 
    num_reviews = num_reviews.replace(',', '')
    num_reviews = int(num_reviews) # convert text into integer
    print('[parse] num_reviews ALL:', num_reviews)

    url_template = url.replace('.html', '-or{}.html')
    print('[parse] url_template:', url_template)

    items = []

    offset = 0

    while(True):
        subpage_url = url_template.format(offset)

        subpage_items = parse_reviews(session, subpage_url)
        if not subpage_items:
            break

        items += subpage_items

        if len(subpage_items) < 5:
            break

        offset += 5

    return items

def get_reviews_ids(soup):

    items = soup.find_all('div', attrs={'data-reviewid': True})

    if items:
        reviews_ids = [x.attrs['data-reviewid'] for x in items][::2]
        print('[get_reviews_ids] data-reviewid:', reviews_ids)
        return reviews_ids
    
def get_more(session, reviews_ids):

    url = 'https://www.tripadvisor.com/OverlayWidgetAjax?Mode=EXPANDED_HOTEL_REVIEWS_RESP&metaReferer=Hotel_Review'

    payload = {
        'reviews': ','.join(reviews_ids), # ie. "577882734,577547902,577300887",
        #'contextChoice': 'DETAIL_HR', # ???
        'widgetChoice': 'EXPANDED_HOTEL_REVIEW_HSX', # ???
        'haveJses': 'earlyRequireDefine,amdearly,global_error,long_lived_global,apg-Hotel_Review,apg-Hotel_Review-in,bootstrap,desktop-rooms-guests-dust-en_US,responsive-calendar-templates-dust-en_US,taevents',
        'haveCsses': 'apg-Hotel_Review-in',
        'Action': 'install',
    }

    soup = post_soup(session, url, payload)

    return soup

def parse_reviews(session, url):
    '''Get all reviews from one page'''

    print('[parse_reviews] url:', url)

    soup =  get_soup(session, url)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    hotel_name = soup.find('h1', id='HEADING').text

    reviews_ids = get_reviews_ids(soup)
    if not reviews_ids:
        return

    soup = get_more(session, reviews_ids)

    if not soup:
        print('[parse_reviews] no soup:', url)
        return

    items = []

    for idx, review in enumerate(soup.find_all('div', class_='reviewSelector')):

        badgets = review.find_all('span', class_='badgetext')
        if len(badgets) > 0:
            contributions = badgets[0].text
        else:
            contributions = '0'

        if len(badgets) > 1:
            helpful_vote = badgets[1].text
        else:
            helpful_vote = '0'
        user_loc = review.select_one('div.userLoc strong')
        if user_loc:
            user_loc = user_loc.text
        else:
            user_loc = ''
            
        bubble_rating = review.select_one('span.ui_bubble_rating')['class']
        bubble_rating = bubble_rating[1].split('_')[-1]
        #print (bubble_rating)
        item = {
            'review_body': review.find('p', class_='partial_entry').text,
            'review_date': review.find('span', class_='ratingDate')['title'], # 'ratingDate' instead of 'relativeDate'
            'review_rating' : bubble_rating
        }

        items.append(item)
        print('\n--- review ---\n')
        for key,val in item.items():
            print(' ', key, ':', val)

    print()

    return items

def write_in_csv(items, filename='results.csv',
                  headers=['hotel name', 'review title', 'review body',
                           'review date', 'contributions', 'helpful vote',
                           'user name' , 'user location', 'rating'],
                  mode='w'):

    print('--- CSV ---')

    with io.open(filename, mode, encoding="utf-8") as csvfile:
        csv_file = csv.DictWriter(csvfile, headers)

        if mode == 'w':
            csv_file.writeheader()

        csv_file.writerows(items)
        
DB_COLUMN   = 'review_body'
DB_COLUMN1 = 'review_date'
DB_COLUMN2 = 'review_rating'

start_urls = [
    'https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali.html',
]

lang = 'id'

headers = [ 
    DB_COLUMN, 
    DB_COLUMN1, 
    DB_COLUMN2,
]

for url in start_urls:

    # get all reviews for 'url' and 'lang'
    items = scrape(url, lang)

    if not items:
        print('No reviews')
    else:
        # write in CSV
        print("else")
        filename = url.split('Reviews-')[1][:-5] + '__' + lang
        print('filename:', filename)
        write_in_csv(items, filename + '.csv', headers, mode='w')
        
#span class="ui_bubble_rating bubble_30" buat cari rating


[parse] url: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali.html?filterLang=id
[parse] num_reviews ALL: 14004
[parse] url_template: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali-or{}.html?filterLang=id
[parse_reviews] url: https://www.tripadvisor.com/Attraction_Review-g297701-d1515658-Reviews-Tegalalang_Rice_Terrace-Ubud_Gianyar_Bali-or0.html?filterLang=id
[get_reviews_ids] data-reviewid: ['638767490', '636845556', '633371086', '632543555', '632106330', '631812013', '631448813', '627627315', '627102465', '623546515']

--- review ---

  review_body : Tempat yang cukup sejuk dan rindang. Cocok untuk menenangkan diri sejenak dari perkotaan. Cocok bagi yang instagramable.
  review_date : December 11, 2018
  review_rating : 50

--- review ---

  review_body : Kerreeeeenn,, keceeee,, indaaahh,,mantaaapp,,pemandangan yg ruuaarr biassaaaaaaahh... Juarraaaa!!! The


--- review ---

  review_body : Jika ingin turun berjalan ke bawah sawah harus melewati cafe-cafe di pinggir jalan.Jalan untuk turun ke bawah sangat sempit harus hati-hati.Disini terdapat toko kecil yang menjual gelato dan jus...perjalanan yang melegakan.
  review_date : October 7, 2018
  review_rating : 50

--- review ---

  review_body :  Bersyukur banget saya lahir dibali,, banyak memiliki tempat wisata yg sangat bagus. Bagus. Pemandangan sawahnya yg hijauspt ini membuat saya tidak ingin meninggalkan tempat ini.I love Bali
  review_date : September 30, 2018
  review_rating : 40

--- review ---

  review_body : Untuk masuk ke tegalalang ini biasanya tidak dipungut biaya, akan tetapi di dekat pintu masuknya biasanya ada pos untuk sumbangan sukarela. Di dalam Tegalalang ini juga terdapat foto booth & juga ayunan raksasa di beberapa spot nya, untuk ayunan raksasa 1 orangnya kena Rp 150.000 untuk kurang lebih 5 kali swing. Saya menyarankan untuk pakai sepatu / sendal yang nyaman karena 


--- review ---

  review_body : sebenernya sih ini ya cuma sawah cuma wisata alam pedesaan di bali tetap punya pecintanya terutama dikalangan turis asing
  review_date : August 15, 2018
  review_rating : 30

--- review ---

  review_body : rice view tempat unik dan keren suasananya juga enak dan asri banget udaranya juga segar untuk joggingtrack juga seru apalagi bareng teman -teman berame rame
  review_date : August 6, 2018
  review_rating : 30

--- review ---

  review_body : Asli keren.padi nya bisa tertata rapi dan asri.pemandangannya juga keren sekali masih alami dan sangat keren
  review_date : July 19, 2018
  review_rating : 30

--- review ---

  review_body : Tempat yg gak asing lagi untuk wisatawan lokal dan asing tempat terasering di pulao bali ini namanya tegal lalang lokasinya di sebelah utara dari desa ubud disini menyediakan pemandangan amparan sawah yang bersusun dari atas ke bawah dengan elokan yg begitu alami cocok buat berfoto buat iklan dan juga rifers pikiran selam

[get_reviews_ids] data-reviewid: ['587289117', '586889965', '585089239', '585053321', '585001518', '583530854', '582811397', '582638048', '582440356', '582110912']

--- review ---

  review_body : Tanaman padi dengan sistem subak di kawasan tegal lalang sangat terkenal dengan keindahannya. banyak wisatawan yang berhenti untuk mengambil gambar, dengan keindahan pemandangan terras,namun sayangnya banyak pedagang acung yang sedikit menggangu untuk menawarkan dagangannya.
  review_date : June 13, 2018
  review_rating : 50

--- review ---

  review_body : View sawah yg bgus sekalli denn pemandangan sawah yg sangat indah sambil menikmati makanan dan minuman sambil.bersantaiDan lbh baik dteng pg to sore nya biar lbh sejuh
  review_date : June 12, 2018
  review_rating : 40

--- review ---

  review_body : Kunjungan ke teras sawah tegagalang sangat recommended karena view yang diberikan sangat indah, namun kunjungan kali ini tidak ada hamparan padi yang menguning karena sudah selesai masa panen

[get_reviews_ids] data-reviewid: ['556614673', '556613826', '555390972', '553512574', '552850984', '551048587', '550140903', '549386065', '548311180', '542645173']

--- review ---

  review_body : Pemandangan persawahan yg sangat indah dan menyegarkan tapi rute ke tempat wisatanya macet total. Perlu oengaturan lalu lintas yang baik
  review_date : January 26, 2018
  review_rating : 40

--- review ---

  review_body : Susunan sawah di sisi jalan Tegallalang sangat indah saya coba untuk berhenti untuk berfono, namun sayangnya sudah banyak tertutup oleh kios kios dan tempat makan, sayang sekali, memang turis termasuk cukup banyak dan lalu lintas padat
  review_date : January 26, 2018
  review_rating : 40

--- review ---

  review_body : Akhirnya tercapai keinginan saya utuk mengunjungi tempat ini. Hamparan sawah berwarna hijau membuat saya merasakan keindahan dan ketenangan yang luar biasa
  review_date : January 22, 2018
  review_rating : 50

--- review ---

  review_body : Apakah di saw

...

[get_reviews_ids] data-reviewid: ['311679143', '309132103', '309121241', '306862492', '306829532', '302684004', '301814035', '293901396', '292819394', '291622755']

--- review ---

  review_body : Di ubud selain udara yang cukup sejuk, hamparan sawah yang hijau sungguh sedap dipandang mata dan menenangkan hati, tertata rapi. Senang bisa menyaksikan langsung pemandangan ini. Elok benar
  review_date : September 18, 2015
  review_rating : 50

--- review ---

  review_body : Melihat terasering sawah yang apik dan hijau, adalah pemandangan yang ditawarkan di Tegallalang Bali.Suasananya damaaaaiii.. Selain itu udaranya yang sejuk ditambah pilihan kopi (jika ingin) sangat membuat hati damai.Jarak tempuh yang tidak cukup jauh dari Kuta, akan sayang dilewatkan jika anda berkunjung ke pulau dewata.
  review_date : September 10, 2015
  review_rating : 50

--- review ---

  review_body : Saya sangat senang berkunjung ke sini karena pemandangannya yang sangat indah. Selain itu, daerah wisata ini p

[get_reviews_ids] data-reviewid: ['289701942', '289062037', '285930957', '282368411', '281611693', '278621414', '274572103', '273487147', '270559455', '268561126']

--- review ---

  review_body : sawah bertangga ini memang indah, terletak di pinggir jalan raya Andong di utara Ubud .Karena tidak ada area parkir,Kendaraan yang parkir dipinggir jalan dan turis yang mengunjungi membuat beban lalu lintas jalan ini menjadi berat.Perlu hati hati kalau sedang naik turun kendaraan atau menyeberang jalan disini.Pemandangan sawah ini tidak istimewa apalagi sewaktu selesai panen . Sawah yang lebih indah bisa dilihat juga di Bali timur dan Bali barat.atau lokasi lain di Ubud seperti ujung Tjampuhan ridge/Sebali.
  review_date : July 16, 2015
  review_rating : 30

--- review ---

  review_body : Tempat yang indah terutama saat padi padi sedang tumbuh, sayangnya area parkirnya terpaksa menggunakan bahu jalan.Trus sekarang sudah banyak resto yang private dan hanya boleh di lalui oleh turis yang makan

[get_reviews_ids] data-reviewid: ['263220642', '262793937', '261174801', '259330047', '257912934', '256647326', '252517430', '251744893', '249665578', '246296296']

--- review ---

  review_body : Tempat wisata yg satu ini juga sangat betgengsi yaitu sawah bersusun.terutama tamu mancanegara yang selalu terkagum melihat tempat ini.dengan pemandangan sawah yang menawan.juga udara yang sepoi2 membuat banyak pelancong yang betlama-lama dantai di tempat ini.pokoknya gak rugi kalau kita mau berkunjung kesana
  review_date : April 2, 2015
  review_rating : 40

--- review ---

  review_body : Jika kita tetap mempertahankan lahan-lahan sawah, lahan tanaman keras, dll dan tidak membiarkannya menjadi "hutan beton".. maka selain kita dapat menikmati keindahan panorama (dapt sebagai terapi mata), juga dapat mencegah terjadinya banjir dan longsor
  review_date : March 30, 2015
  review_rating : 30

--- review ---

  review_body : Seperti lukisan, itu adalah kesan saya melihat indahnya lokasi ini. Ba

[get_reviews_ids] data-reviewid: ['246041687', '245917519', '245696381', '245595459', '244897615', '243774233', '239596920', '239357132', '237346156', '236022484']

--- review ---

  review_body : Hawa sejuk dan suasana pedesaan bali sangat kental terasa...selama perjalanan dari ubud disuguhi desa wisata kerajinan tangan dengan nuansa seni tinggi khas bali....bagi anak anak bisa mengenal langsung teras siring dan subak khas pertanian Bali. Menajubkan
  review_date : December 27, 2014
  review_rating : 40

--- review ---

  review_body : Mungkin namanya terlalu dibesar-besarkan dari keadaan sesungguhnya. Sawah dan pemandangannya memang cantik, tapi tempatnya sendiri sudah ramai dengan deretan warung yang memenuhi pinggir jalan, jadi untuk dari kendaraan tak terlihat jelas pemandangan cantik ini. 
  review_date : December 26, 2014
  review_rating : 30

--- review ---

  review_body : Pemandangan sawah ciri khas masyarakat bali.. Dapat dilihat sambil menikmati makanan minuman dari resto/w

[get_reviews_ids] data-reviewid: ['234883463', '222545578', '216015641', '214603883', '211040617', '208984376', '205198085', '196367101', '190900900', '129449186']

--- review ---

  review_body : Ceking-Ceking Tegalalang dikenal sebagai objek dengan pemandangan terasering sawah yang indah. Berlokasi sekitar 30 menit berkendara dari Ubud dan sekitar 1,5 jam dari Kuta.Disini kita akan menyaksikan pemandangan sawah dengan teraseringnya yang terlihat indah dengan berbagai lekukannya. Jika musim tanam tiba kita juga bisa menyaksikan bagai mana proses penanaman makanan pokok yang selama ini hanya bisa kita nikmati tetapi tidak tau bagaimana keunikan proses penanamannya.Jika berkunjung ke Ceking Tegalalang pada jam makan siang, tentunya sangat tepat menikmti makan siang di sini dengan suasana nyaman sambil menyaksikan keindahan sawahnya. Ada beberapa restaurant yang bisa kita coba disini yang menawarkan masakan khas indonesia atau masakan khas bali dengan view sawah.Jadi jika anda berlibur k